In [1]:
import io
import re
import numpy as np
import pandas as pd
import numpy.linalg as lin
import warnings 
from gensim.models import Word2Vec
import pickle
from sklearn.cluster import KMeans
from indic_transliteration import sanscript
from indic_transliteration.sanscript import transliterate
warnings.filterwarnings('ignore')

In [2]:
stopwords=['ଦେଇଛନ୍ତି', 'ଲେଖାଏଁ', 'ଜଣେ', 'ଏହା', 'ତେଣୁ', 'ମିଳିଥାଏ', 'ପାଇଁ', 'ନେଉଛନ୍ତି', 'ଯୋଗୁଁ', 'ଏପର୍ଯ୍ୟନ୍ତ', 'ଏଭଳି', 'କରୁଛି', 'ଓ', 'ଯାଏଁ','ହୋଇଛନ୍ତି', 'କି', 'କରାଯିବା', 'ପରେ', 'ଏହି', 'ଏବଂ', 'ଜଣ', 'ଥିବା','ହୋଇଥିଲା', 'ତେବେ', 'ଆଜି', 'ଜଣଙ୍କ', 'ଏଥି', 'ଗତ', 'ହୋଇଥିଲେ',
       'ଦିନ ଧରି', 'ହେଉଥିବା', 'ଦିନ ତଳେ', 'ଯୋଗେ', 'ବୋଲି', 'ଜଣାପଡ଼ିଛି',
       'ଦ୍ବାରା', 'କରି', 'ଯାଇ', 'ଏ ନେଇ', 'ଚାଲୁ', 'ରହିଛି', 'ତାରିଖ',
       'ମିଳିଛି', 'ବର୍ଷୀୟ', 'ଦିନ ତଳେ', 'ସହ', 'ଆସିଛନ୍ତି', 'ମଧ୍ୟ', 'କେଉଁ',
       'ହୋଇଯାଇଛି', 'ନେଇ ଯାଇଛି', 'କାମ', 'କରିଛି', 'ହେବା', 'ପରେ', 'ଏବେ',
       'ହୋଇଛି', 'ତେବେ', 'ରଖି', 'ନିଆଯାଇଥିବା', 'ପର୍ଯ୍ୟନ୍ତ', 'କରିଛନ୍ତି',
       'ଉପରେ', 'ଦେଲେ', 'ଥର', 'ଏବଂ', 'ଆଉ']
alphabets=['ଅ','ଆ','ଇ','ଈ','ଉ','ଊ','ଋ','ଌ','ଏ','ଐ','ଓ','ଔ','କ','ଖ','ଗ','ଘ','ଙ','ଚ','ଛ','ଜ','ଝ','ଞ','ଟ','ଠ','ଡ','ଢ','ଣ',
 'ତ','ଥ','ଦ','ଧ','ନ','ପ','ଫ','ବ','ଭ','ମ','ଯ','ର','ଲ','ଳ','ଵ','ଶ','ଷ','ସ','ହ','ଡ଼','ଢ଼','ୟ','ୠ','ୡ','ୱ']
matra=['ଁ','ଂ','ଃ','଼','ା','ି','ୀ','ୁ','ୂ','ୃ','ୄ','େ','ୈ','ୋ','ୌ','୍','ୢ','ୣ']

In [3]:
delimiters=["।","?","!"]

In [4]:
fasttext=Word2Vec.load('fasttext/or-d100-m2-fasttext.model')

In [5]:
file=open('sentence.pkl','rb')
verse_embeddings_sentence=pickle.load(file)
file.close()
file=open('max.pkl','rb')
verse_embeddings_max=pickle.load(file)
file.close()
file=open('mean.pkl','rb')
verse_embeddings_mean=pickle.load(file)
file.close()

In [6]:
file=open('whole.pkl','rb')
verse_embeddings_whole=pickle.load(file)
file.close()

In [7]:
def o2h(text):
    return transliterate(text, sanscript.ORIYA, sanscript.DEVANAGARI)

In [8]:
def remove_special_characters(text):
    string=''
    for i in text:
        if i in alphabets+matra+delimiters:
            string=string+i
        else:
            string=string+" "
    return " ".join(string.split())
def sent_tokenize(text):
    text=remove_special_characters(text)
    regexPattern = '|'.join(map(re.escape, delimiters))
    sentences=re.split(regexPattern,text)
    return [sent.strip() for sent in sentences if len(sent.strip())>0]

In [9]:
def word_embedding(word):
    try:
        return fasttext.wv[o2h(word)]
    except:
        return np.array([0.0]*100)

In [10]:
def cosine_similarity(a,b):
    try:
        return a.dot(b)/(lin.norm(a)*lin.norm(b))
    except:
        return -1.0

In [11]:
def ed(a,b):
    return lin.norm(a-b)

In [12]:
def sentence_embedding(sentence):
    sentence=remove_special_characters(sentence)
    words=sentence.split()
    words=[word  for word in words if word not in stopwords]
    if len(words)>0:
        sentence_embedding=[word_embedding(word) for word in words]
        return np.array(list(map(lambda x: sum(x)/len(x), zip(*sentence_embedding))))
    return np.array([0.0]*100)

In [13]:
data=pd.read_csv('../odia.csv')

In [14]:
query='ଆନନ୍ଦ ଖୋଜିବା ମନର ସ୍ୱଭାବ ଅଟେ ଏବଂ ଭୌତିକ ପ୍ରକୃତିରୁ ଉତ୍ପନ୍ନ ହୋଇଥିବା ଯୋଗୁଁ, ଭୌତିକ ସୁଖ ପ୍ରତି ସେ ସ୍ୱତଃ ଆକୃଷ୍ଟ ହୋଇଥାଏ । ଆମେ ଯଦି ଆମ ମନର ପ୍ରବୃତ୍ତି ପଛରେ ଗୋଡ଼ାଇବା, ଭୌତିକ ଚେତନାର ଗଭୀରତମ ପ୍ରଦେଶକୁ ଆମର ପତନ ହୋଇଯିବ । ଏହି ଅଧୋଗତିକୁ ପ୍ରତିହତ କରିବାର ଉପାୟ ହେଉଛି, ବୁଦ୍ଧିଦ୍ୱାରା ମନକୁ ନିୟନ୍ତ୍ରଣ କରିବା । ସେଥିପାଇଁ ବୁଦ୍ଧିକୁ ପ୍ରକୃତ ଜ୍ଞାନ ଦ୍ୱାରା ସମୃଦ୍ଧ କରିବା ଆବଶ୍ୟକ ।ଯେଉଁମାନେ ଭଗବାନଙ୍କୁ ସମସ୍ତ ପ୍ରାଣୀଙ୍କ ମଧ୍ୟରେ ସ୍ଥିତ ପରମାତ୍ମା ରୂପରେ ଦେଖିବାର ଅଭ୍ୟାସ କରିଥାନ୍ତି, ସେମାନେ ଏହି ଜ୍ଞାନରେ ସ୍ଥିତ ରହନ୍ତି । ସେମାନେ ଅନ୍ୟମାନଙ୍କ ସହିତ ସମ୍ପର୍କରୁ ବ୍ୟକ୍ତିଗତ ଲାଭ ବା ସୁଖ ଖୋଜନ୍ତି ନାହିଁ । ଅନ୍ୟମାନେ ତାଙ୍କ ପ୍ରତି କରିଥିବା ଉପକାର ପାଇଁ ସେମାନେ ତାଙ୍କଠାରେ ଆସକ୍ତ ହୁଅନ୍ତି ନାହିଁ କିମ୍ବା ଅନ୍ୟମାନେ ତାଙ୍କ ପ୍ରତି କରିଥିବା କ୍ଷତି ପାଇଁ ସେମାନେ ତାଙ୍କୁ ଘୃଣା କରନ୍ତି ନାହିଁ । ବରଂ, ସମସ୍ତଙ୍କୁ ଭଗବାନଙ୍କର ଅଂଶ ମନେ କରି, ସେମାନେ ସମସ୍ତଙ୍କ ପ୍ରତି ସମ୍ମାନ ଓ ସେବା ଭାବର ଏକ ସୁସ୍ଥ ମନୋଭାବ ପୋଷଣ କରନ୍ତି । ସମସ୍ତଙ୍କ ଠାରେ ଭଗବାନଙ୍କ ଉପସ୍ଥିତି ଅନୁଭବ କରିବା ଯୋଗୁଁ ସେମାନେ ଅନ୍ୟମାନଙ୍କୁ ଦୁର୍ବ୍ୟବହାର କରିବା, ଠକିବା କିମ୍ବା ଅପମାନିତ କରିବା ଠାରୁ କ୍ଷାନ୍ତ ରହନ୍ତି । ତା’ ସହିତ, ଜାତୀୟତା, ଧର୍ମ, ଜାତି, ଲିଙ୍ଗ, ପଦବୀ ଏବଂ ବର୍ଣ୍ଣ ଇତ୍ୟାଦି ମନୁଷ୍ୟକୃତ ପ୍ରଭେଦ ତାଙ୍କ ପାଇଁ ଅର୍ଥହୀନ ହୋଇଥାଏ । ଅତଏବ, ଭଗବାନଙ୍କୁ ସବୁ ପ୍ରାଣୀଙ୍କଠାରେ ଦେଖି ସେମାନେ ତାଙ୍କ ମନକୁ ଉଚ୍ଚସ୍ଥିତିକୁ ନେଇଯାନ୍ତି ଏବଂ ଶେଷରେ ଚରମ ଲକ୍ଷ୍ୟକୁ ପ୍ରାପ୍ତ କରନ୍ତି ।'
query_embedding=sentence_embedding(query)
query_embedding_list=query_embedding.reshape(1,-1).astype(np.float64)

# Max Strategy

## Cosine Similarity

In [15]:
def max_cosine(query_embedding,counter=10):
    scoring=[cosine_similarity(query_embedding,i) for i in verse_embeddings_max]
    verse_index=np.argsort(scoring)[-1::-1][:counter]
    return verse_index

In [16]:
verse_index=max_cosine(query_embedding)

In [17]:
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
363,9,33,किं पुनर्ब्राह्मणाः पुण्या भक्ता राजर्षयस्तथा।...,ଯଦି ଅତ୍ୟନ୍ତ ପାପୀମାନେ ମଧ୍ୟ ଭକ୍ତି ମାର୍ଗରେ ସଫଳତା ...
573,16,11,चिन्तामपरिमेयां च प्रलयान्तामुपाश्रिताः।कामोपभ...,ବସ୍ତୁବାଦୀମାନେ ଅଧିକାଂଶତଃ ଏହି କାରଣରୁ ଆଧ୍ୟାତ୍ମିକ ...
139,3,28,तत्त्ववित्तु महाबाहो गुणकर्मविभागयोः।\n\nगुणा ...,"ପୂର୍ବବର୍ତ୍ତୀ ଶ୍ଲୋକରେ ଉଲ୍ଲେଖ କରାଯାଇଛି, ଅହଂକାର ବ..."
118,3,7,यस्त्विन्द्रियाणि मनसा नियम्यारभतेऽर्जुन।\n\nक...,ଏହି ଶ୍ଲୋକରେ ବ୍ୟବହୃତ କର୍ମଯୋଗ ଶବ୍ଦରେ ମୁଖ୍ୟତଃ ଦୁଇ...
292,7,20,कामैस्तैस्तैर्हृतज्ञानाः प्रपद्यन्तेऽन्यदेवताः...,ପରମ ପୁରୁଷ ଶ୍ରୀକୃଷ୍ଣ ସୃଷ୍ଟିର ସବୁକିଛିର ଆଧାର ଅଟନ୍...
259,6,34,चञ्चलं हि मनः कृष्ण प्रमाथि बलवद्दृढम्।\n\nतस्...,"ଅର୍ଜୁନ, ଆମ ସମସ୍ତଙ୍କୁ ଲକ୍ଷ୍ୟ କରି, ଏହି ଉପଦ୍ରବୀ ମ..."
253,6,28,युञ्जन्नेवं सदाऽऽत्मानं योगी विगतकल्मषः।\n\nसु...,ଆନନ୍ଦକୁ ଚାରି ଭାଗରେ ବିଭକ୍ତ କରାଯାଇପାରେ:ସାତ୍ତ୍ୱିକ...
145,3,34,इन्द्रियस्येन्द्रियस्यार्थे रागद्वेषौ व्यवस्थि...,ପୂର୍ବରୁ ଯଦିଓ ଶ୍ରୀକୃଷ୍ଣ ଗୁରୁତ୍ୱାରୋପଣ କରିଥିଲେ ଯେ...
647,18,33,धृत्या यया धारयते मनःप्राणेन्द्रियक्रियाः।योगे...,ବାଧାବିଘ୍ନ ସତ୍ତ୍ୱେ ନିଜ ମାର୍ଗରେ ଅଟଳ ରହିବା ପାଇଁ ମ...
566,16,4,दम्भो दर्पोऽभिमानश्च क्रोधः पारुष्यमेव च।अज्ञा...,ଶ୍ରୀକୃଷ୍ଣ ବର୍ତ୍ତମାନ ଆସୁରିକ ପ୍ରବୃତ୍ତିଯୁକ୍ତ ବ୍ୟକ...


## Euclidean Distance

In [18]:
def max_ed(query_embedding,counter=10):
    scoring=[ed(query_embedding,i) for i in verse_embeddings_max]
    verse_index=np.argsort(scoring)[:counter]
    return verse_index

In [19]:
verse_index=max_ed(query_embedding)

In [20]:
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
363,9,33,किं पुनर्ब्राह्मणाः पुण्या भक्ता राजर्षयस्तथा।...,ଯଦି ଅତ୍ୟନ୍ତ ପାପୀମାନେ ମଧ୍ୟ ଭକ୍ତି ମାର୍ଗରେ ସଫଳତା ...
573,16,11,चिन्तामपरिमेयां च प्रलयान्तामुपाश्रिताः।कामोपभ...,ବସ୍ତୁବାଦୀମାନେ ଅଧିକାଂଶତଃ ଏହି କାରଣରୁ ଆଧ୍ୟାତ୍ମିକ ...
139,3,28,तत्त्ववित्तु महाबाहो गुणकर्मविभागयोः।\n\nगुणा ...,"ପୂର୍ବବର୍ତ୍ତୀ ଶ୍ଲୋକରେ ଉଲ୍ଲେଖ କରାଯାଇଛି, ଅହଂକାର ବ..."
118,3,7,यस्त्विन्द्रियाणि मनसा नियम्यारभतेऽर्जुन।\n\nक...,ଏହି ଶ୍ଲୋକରେ ବ୍ୟବହୃତ କର୍ମଯୋଗ ଶବ୍ଦରେ ମୁଖ୍ୟତଃ ଦୁଇ...
512,13,32,अनादित्वान्निर्गुणत्वात्परमात्मायमव्ययः।शरीरस्...,"ଭଗବାନ ସମସ୍ତ ପ୍ରାଣୀଙ୍କ ହୃଦୟରେ, ପରମାତ୍ମା ରୂପରେ ନ..."
76,2,37,हतो वा प्राप्स्यसि स्वर्गं जित्वा वा भोक्ष्यसे...,୨.୩୧ ଶ୍ଳୋକ ଠାରୁ ଆରମ୍ଭ କରି ଶ୍ରୀକୃଷ୍ଣ ଏ ପର୍ଯ୍ୟନ୍...
468,12,7,तेषामहं समुद्धर्ता मृत्युसंसारसागरात्।भवामि नच...,ଶ୍ରୀକୃଷ୍ଣ ପୁନର୍ବାର କହୁଛନ୍ତି ଯେ ତାଙ୍କର ଭକ୍ତମାନେ...
467,12,6,ये तु सर्वाणि कर्माणि मयि संन्यस्य मत्पराः।अनन...,ଶ୍ରୀକୃଷ୍ଣ ପୁନର୍ବାର କହୁଛନ୍ତି ଯେ ତାଙ୍କର ଭକ୍ତମାନେ...
253,6,28,युञ्जन्नेवं सदाऽऽत्मानं योगी विगतकल्मषः।\n\nसु...,ଆନନ୍ଦକୁ ଚାରି ଭାଗରେ ବିଭକ୍ତ କରାଯାଇପାରେ:ସାତ୍ତ୍ୱିକ...
636,18,22,यत्तु कृत्स्नवदेकस्मिन्कार्ये सक्तमहैतुकम्।अतत...,"ତମୋଗୁଣ ପ୍ରଭାବରେ ବୁଦ୍ଧି ଯେତେବେଳେ ଶିଥିଳ ହୋଇଯାଏ, ..."


# Mean Pooling

## Cosine Similarity

In [21]:
def mean_cosine(query_embedding,counter=10):
    scoring=[cosine_similarity(query_embedding,i) for i in verse_embeddings_mean]
    verse_index=np.argsort(scoring)[-1::-1][:counter]
    return verse_index

In [22]:
verse_index=mean_cosine(query_embedding)

In [23]:
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
509,13,29,समं पश्यन्हि सर्वत्र समवस्थितमीश्वरम्।न हिनस्त...,ଆନନ୍ଦ ଖୋଜିବା ମନର ସ୍ୱଭାବ ଅଟେ ଏବଂ ଭୌତିକ ପ୍ରକୃତିର...
573,16,11,चिन्तामपरिमेयां च प्रलयान्तामुपाश्रिताः।कामोपभ...,ବସ୍ତୁବାଦୀମାନେ ଅଧିକାଂଶତଃ ଏହି କାରଣରୁ ଆଧ୍ୟାତ୍ମିକ ...
474,12,13,अद्वेष्टा सर्वभूतानां मैत्रः करुण एव च।निर्ममो...,ତାଙ୍କ ସାକାର ସ୍ୱରୂପର ଭକ୍ତିକୁ ସର୍ବୋତ୍କୃଷ୍ଟ କହିବ...
475,12,14,सन्तुष्टः सततं योगी यतात्मा दृढनिश्चयः।मय्यर्प...,ତାଙ୍କ ସାକାର ସ୍ୱରୂପର ଭକ୍ତିକୁ ସର୍ବୋତ୍କୃଷ୍ଟ କହିବ...
476,12,15,यस्मान्नोद्विजते लोको लोकान्नोद्विजते च यः।हर्...,ଆତ୍ମା ସ୍ୱଭାବତଃ ପବିତ୍ର ଏବଂ ବିଶୁଦ୍ଧ ଅଟେ । କିନ୍ତୁ...
219,5,23,शक्नोतीहैव यः सोढुं प्राक्शरीरविमोक्षणात्।\n\n...,ମନୁଷ୍ୟ ଶରୀର ଆତ୍ମାକୁ ତା’ର ଭଗବତ୍ ପ୍ରାପ୍ତି ରୂପୀ ଚ...
578,16,16,अनेकचित्तविभ्रान्ता मोहजालसमावृताः।प्रसक्ताः क...,ଅହଂକାରର ପ୍ରଭାବରେ ବ୍ୟକ୍ତି ନିଜକୁ ମନ ମନେକରି ତା’ର ...
471,12,10,अभ्यासेऽप्यसमर्थोऽसि मत्कर्मपरमो भव।मदर्थमपि क...,ଭଗବାନଙ୍କର ସ୍ମରଣ କରିବା ପାଇଁ ଉପଦେଶ ଦେବା ତାହାକୁ କ...
565,16,3,तेजः क्षमा धृतिः शौचमद्रोहो नातिमानिता।\n\nभवन...,ଶ୍ରୀକୃଷ୍ଣ ଏଠାରେ ସାଧୁ ଚରିତ୍ରର ଛବିଶଗୋଟି ଗୁଣ ବର୍ଣ...
564,16,2,अहिंसा सत्यमक्रोधस्त्यागः शान्तिरपैशुनम्।दया भ...,ଶ୍ରୀକୃଷ୍ଣ ଏଠାରେ ସାଧୁ ଚରିତ୍ରର ଛବିଶଗୋଟି ଗୁଣ ବର୍ଣ...


## Euclidean Distance

In [24]:
def mean_ed(query_embedding,counter=10):
    scoring=[ed(query_embedding,i) for i in verse_embeddings_mean]
    verse_index=np.argsort(scoring)[:counter]
    return verse_index

In [25]:
verse_index=mean_ed(query_embedding)

In [26]:
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
509,13,29,समं पश्यन्हि सर्वत्र समवस्थितमीश्वरम्।न हिनस्त...,ଆନନ୍ଦ ଖୋଜିବା ମନର ସ୍ୱଭାବ ଅଟେ ଏବଂ ଭୌତିକ ପ୍ରକୃତିର...
573,16,11,चिन्तामपरिमेयां च प्रलयान्तामुपाश्रिताः।कामोपभ...,ବସ୍ତୁବାଦୀମାନେ ଅଧିକାଂଶତଃ ଏହି କାରଣରୁ ଆଧ୍ୟାତ୍ମିକ ...
475,12,14,सन्तुष्टः सततं योगी यतात्मा दृढनिश्चयः।मय्यर्प...,ତାଙ୍କ ସାକାର ସ୍ୱରୂପର ଭକ୍ତିକୁ ସର୍ବୋତ୍କୃଷ୍ଟ କହିବ...
474,12,13,अद्वेष्टा सर्वभूतानां मैत्रः करुण एव च।निर्ममो...,ତାଙ୍କ ସାକାର ସ୍ୱରୂପର ଭକ୍ତିକୁ ସର୍ବୋତ୍କୃଷ୍ଟ କହିବ...
476,12,15,यस्मान्नोद्विजते लोको लोकान्नोद्विजते च यः।हर्...,ଆତ୍ମା ସ୍ୱଭାବତଃ ପବିତ୍ର ଏବଂ ବିଶୁଦ୍ଧ ଅଟେ । କିନ୍ତୁ...
219,5,23,शक्नोतीहैव यः सोढुं प्राक्शरीरविमोक्षणात्।\n\n...,ମନୁଷ୍ୟ ଶରୀର ଆତ୍ମାକୁ ତା’ର ଭଗବତ୍ ପ୍ରାପ୍ତି ରୂପୀ ଚ...
578,16,16,अनेकचित्तविभ्रान्ता मोहजालसमावृताः।प्रसक्ताः क...,ଅହଂକାରର ପ୍ରଭାବରେ ବ୍ୟକ୍ତି ନିଜକୁ ମନ ମନେକରି ତା’ର ...
490,13,10,असक्ितरनभिष्वङ्गः पुत्रदारगृहादिषु।नित्यं च सम...,କ୍ଷେତ୍ର ଏବଂ କ୍ଷେତ୍ରଜ୍ଞ ବିଷୟରେ ଜ୍ଞାନ ହାସଲ କରିବା...
491,13,11,मयि चानन्ययोगेन भक्ितरव्यभिचारिणी।विविक्तदेशसे...,କ୍ଷେତ୍ର ଏବଂ କ୍ଷେତ୍ରଜ୍ଞ ବିଷୟରେ ଜ୍ଞାନ ହାସଲ କରିବା...
492,13,12,अध्यात्मज्ञाननित्यत्वं तत्त्वज्ञानार्थदर्शनम्।...,କ୍ଷେତ୍ର ଏବଂ କ୍ଷେତ୍ରଜ୍ଞ ବିଷୟରେ ଜ୍ଞାନ ହାସଲ କରିବା...


# Line Matching

## Cosine Similarity

In [27]:
def line_cosine(query_embeddings,counter=10):
    scoring=[]
    for embeddings in verse_embeddings_sentence:
        score=[cosine_similarity(query_embedding,embed) for embed in embeddings]
        scoring.append(max(score))
    verse_index=np.argsort(scoring)[-1::-1][:counter]
    return verse_index

In [28]:
verse_index=line_cosine(query_embedding)
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
329,8,27,नैते सृती पार्थ जानन्योगी मुह्यति कश्चन।\n\nतस...,"ଯୋଗୀମାନେ ସେହି ପ୍ରକାର ସାଧକ ଅଟନ୍ତି, ଯେଉଁମାନେ ନିଜ..."
361,9,31,क्षिप्रं भवति धर्मात्मा शश्वच्छान्तिं निगच्छति...,"କେବଳ ଉଚିତ୍ ସଂକଳ୍ପ କଲେ, ଜଣେ ଭକ୍ତକୁ ଧର୍ମାତ୍ମା କୁ..."
491,13,11,मयि चानन्ययोगेन भक्ितरव्यभिचारिणी।विविक्तदेशसे...,କ୍ଷେତ୍ର ଏବଂ କ୍ଷେତ୍ରଜ୍ଞ ବିଷୟରେ ଜ୍ଞାନ ହାସଲ କରିବା...
490,13,10,असक्ितरनभिष्वङ्गः पुत्रदारगृहादिषु।नित्यं च सम...,କ୍ଷେତ୍ର ଏବଂ କ୍ଷେତ୍ରଜ୍ଞ ବିଷୟରେ ଜ୍ଞାନ ହାସଲ କରିବା...
489,13,9,इन्द्रियार्थेषु वैराग्यमनहङ्कार एव च।जन्ममृत्य...,କ୍ଷେତ୍ର ଏବଂ କ୍ଷେତ୍ରଜ୍ଞ ବିଷୟରେ ଜ୍ଞାନ ହାସଲ କରିବା...
488,13,8,अमानित्वमदम्भित्वमहिंसा क्षान्तिरार्जवम्।आचार्...,କ୍ଷେତ୍ର ଏବଂ କ୍ଷେତ୍ରଜ୍ଞ ବିଷୟରେ ଜ୍ଞାନ ହାସଲ କରିବା...
492,13,12,अध्यात्मज्ञाननित्यत्वं तत्त्वज्ञानार्थदर्शनम्।...,କ୍ଷେତ୍ର ଏବଂ କ୍ଷେତ୍ରଜ୍ଞ ବିଷୟରେ ଜ୍ଞାନ ହାସଲ କରିବା...
524,14,9,सत्त्वं सुखे सञ्जयति रजः कर्मणि भारत।ज्ञानमावृ...,"ବ୍ୟକ୍ତି ସଦ୍‌ଗୁଣ ଯୁକ୍ତ ହେଲେ, ଭୌତିକ ଜଗତର କଷ୍ଟ କମ..."
341,9,11,अवजानन्ति मां मूढा मानुषीं तनुमाश्रितम्।\n\nपर...,ଉତ୍ତମ ଶିକ୍ଷକମାନେ ବେଳେବେଳେ ତାଙ୍କ ଛାତ୍ରମାନଙ୍କ ପ୍...
468,12,7,तेषामहं समुद्धर्ता मृत्युसंसारसागरात्।भवामि नच...,ଶ୍ରୀକୃଷ୍ଣ ପୁନର୍ବାର କହୁଛନ୍ତି ଯେ ତାଙ୍କର ଭକ୍ତମାନେ...


## Euclidean Distance

In [29]:
def line_ed(query_embeddings,counter=10):
    scoring=[]
    for embeddings in verse_embeddings_sentence:
        score=[ed(query_embedding,embed) for embed in embeddings]
        scoring.append(max(score))
    verse_index=np.argsort(scoring)[:counter]
    return verse_index

In [30]:
verse_index=line_ed(query_embedding)
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
512,13,32,अनादित्वान्निर्गुणत्वात्परमात्मायमव्ययः।शरीरस्...,"ଭଗବାନ ସମସ୍ତ ପ୍ରାଣୀଙ୍କ ହୃଦୟରେ, ପରମାତ୍ମା ରୂପରେ ନ..."
636,18,22,यत्तु कृत्स्नवदेकस्मिन्कार्ये सक्तमहैतुकम्।अतत...,"ତମୋଗୁଣ ପ୍ରଭାବରେ ବୁଦ୍ଧି ଯେତେବେଳେ ଶିଥିଳ ହୋଇଯାଏ, ..."
8,1,11,अयनेषु च सर्वेषु यथाभागमवस्थिताः।\n\nभीष्ममेवा...,ଦୁର୍ଯ୍ୟୋଧନ ପିତାମହ ଭୀଷ୍ମଙ୍କର ଅଭେଦ୍ୟ କ୍ଷମତାକୁ ତ...
484,13,4,तत्क्षेत्रं यच्च यादृक् च यद्विकारि यतश्च यत्।...,ବର୍ତ୍ତମାନ ଶ୍ରୀକୃଷ୍ଣ ନିଜେ ଅନେକ ପ୍ରଶ୍ନ ଉପସ୍ଥାପନ ...
140,3,29,प्रकृतेर्गुणसम्मूढाः सज्जन्ते गुणकर्मसु।\n\nता...,ଏଠାରେ ପ୍ରଶ୍ନ ଉଠିପାରେ ଯେ.ଯଦି ଆତ୍ମା ଗୁଣ ଏବଂ ସେ...
649,18,35,यया स्वप्नं भयं शोकं विषादं मदमेव च।न विमुञ्चत...,ବୁଦ୍ଧିହୀନ ଏବଂ ମୁର୍ଖମାନଙ୍କ ଠାରେ ମଧ୍ୟ ଦୃଢ଼ ଇଚ୍ଛାଶ...
428,11,22,रुद्रादित्या वसवो ये च साध्या\n\nविश्वेऽश्िवनौ...,ଏହି ସବୁ ବ୍ୟକ୍ତିତ୍ୱ ସେମାନଙ୍କର ପଦବୀ ଭଗବାନଙ୍କ ଶକ୍...
422,11,16,अनेकबाहूदरवक्त्रनेत्रं\n\nपश्यामि त्वां सर्वतो...,ଅର୍ଜୁନ ଦୁଇଟି ଶବ୍ଦ ବ୍ୟବହାର କରିଛନ୍ତି - ବିଶ୍ୱେଶ୍ୱ...
407,11,1,अर्जुन उवाच\n\nमदनुग्रहाय परमं गुह्यमध्यात्मसं...,ଶ୍ରୀକୃଷ୍ଣଙ୍କର ସମସ୍ତ ଐଶ୍ୱର୍ଯ୍ୟ ଏବଂ ତାଙ୍କର ସର୍ବେ...
229,6,4,यदा हि नेन्द्रियार्थेषु न कर्मस्वनुषज्जते।\n\n...,ମନ ଯେତେବେଳେ ଯୋଗରେ ଭଗବାନଙ୍କ ସହିତ ସଂଯୁକ୍ତ ହୋଇଯାଏ...


# Whole Part

## Cosine Similarity

In [31]:
def whole_cosine(query_embedding,counter=10):
    scoring=[cosine_similarity(query_embedding,i) for i in verse_embeddings_whole]
    verse_index=np.argsort(scoring)[-1::-1][:counter]
    return verse_index

In [32]:
verse_index=whole_cosine(query_embedding)
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
509,13,29,समं पश्यन्हि सर्वत्र समवस्थितमीश्वरम्।न हिनस्त...,ଆନନ୍ଦ ଖୋଜିବା ମନର ସ୍ୱଭାବ ଅଟେ ଏବଂ ଭୌତିକ ପ୍ରକୃତିର...
475,12,14,सन्तुष्टः सततं योगी यतात्मा दृढनिश्चयः।मय्यर्प...,ତାଙ୍କ ସାକାର ସ୍ୱରୂପର ଭକ୍ତିକୁ ସର୍ବୋତ୍କୃଷ୍ଟ କହିବ...
474,12,13,अद्वेष्टा सर्वभूतानां मैत्रः करुण एव च।निर्ममो...,ତାଙ୍କ ସାକାର ସ୍ୱରୂପର ଭକ୍ତିକୁ ସର୍ବୋତ୍କୃଷ୍ଟ କହିବ...
578,16,16,अनेकचित्तविभ्रान्ता मोहजालसमावृताः।प्रसक्ताः क...,ଅହଂକାରର ପ୍ରଭାବରେ ବ୍ୟକ୍ତି ନିଜକୁ ମନ ମନେକରି ତା’ର ...
476,12,15,यस्मान्नोद्विजते लोको लोकान्नोद्विजते च यः।हर्...,ଆତ୍ମା ସ୍ୱଭାବତଃ ପବିତ୍ର ଏବଂ ବିଶୁଦ୍ଧ ଅଟେ । କିନ୍ତୁ...
573,16,11,चिन्तामपरिमेयां च प्रलयान्तामुपाश्रिताः।कामोपभ...,ବସ୍ତୁବାଦୀମାନେ ଅଧିକାଂଶତଃ ଏହି କାରଣରୁ ଆଧ୍ୟାତ୍ମିକ ...
221,5,25,लभन्ते ब्रह्मनिर्वाणमृषयः क्षीणकल्मषाः।\n\nछिन...,"ପୂର୍ବ ଶ୍ଲୋକରେ, ଶ୍ରୀକୃଷ୍ଣ ସେହି ମହାପୁରୁଷମାନଙ୍କ ବ..."
492,13,12,अध्यात्मज्ञाननित्यत्वं तत्त्वज्ञानार्थदर्शनम्।...,କ୍ଷେତ୍ର ଏବଂ କ୍ଷେତ୍ରଜ୍ଞ ବିଷୟରେ ଜ୍ଞାନ ହାସଲ କରିବା...
489,13,9,इन्द्रियार्थेषु वैराग्यमनहङ्कार एव च।जन्ममृत्य...,କ୍ଷେତ୍ର ଏବଂ କ୍ଷେତ୍ରଜ୍ଞ ବିଷୟରେ ଜ୍ଞାନ ହାସଲ କରିବା...
490,13,10,असक्ितरनभिष्वङ्गः पुत्रदारगृहादिषु।नित्यं च सम...,କ୍ଷେତ୍ର ଏବଂ କ୍ଷେତ୍ରଜ୍ଞ ବିଷୟରେ ଜ୍ଞାନ ହାସଲ କରିବା...


## Euclidean Distance

In [33]:
def whole_ed(query_embedding,counter=10):
    scoring=[cosine_similarity(query_embedding,i) for i in verse_embeddings_whole]
    verse_index=np.argsort(scoring)[:counter]
    return verse_index

In [34]:
verse_index=whole_ed(query_embedding)
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
391,10,27,उच्चैःश्रवसमश्वानां विद्धि माममृतोद्भवम्।\n\nऐ...,ସବୁ ବିଭାଗର ଶ୍ରେଷ୍ଠମାନଙ୍କୁ ଶ୍ରୀକୃଷ୍ଣ ତାଙ୍କର ଐଶ୍...
401,10,37,वृष्णीनां वासुदेवोऽस्मि पाण्डवानां धनंजयः।\n\n...,ବୃଷ୍ଣୀବଂଶରେ ବସୁଦେବଙ୍କ ପୁତ୍ର ରୂପରେ ଶ୍ରୀକୃଷ୍ଣ ପୃ...
394,10,30,प्रह्लादश्चास्मि दैत्यानां कालः कलयतामहम्।\n\n...,ପରାକ୍ରମଶାଳୀ ଦୈତ୍ୟରାଜ ହିରଣ୍ୟ-କଶିପୁଙ୍କ ପୁତ୍ର ଭାବ...
689,18,75,व्यासप्रसादाच्छ्रुतवानेतद्गुह्यमहं परम्।योगं य...,"ଶ୍ରୀକୃଷ୍ଣ ଦ୍ୱୈପାୟନ ବ୍ୟାସଦେବ, ଯିଏ ମହର୍ଷି ବେଦବ୍ୟ..."
449,11,43,पितासि लोकस्य चराचरस्य\n\nत्वमस्य पूज्यश्च गुर...,"ଅର୍ଜୁନ କହୁଛନ୍ତି ଯେ, ଶ୍ରୀକୃଷ୍ଣ ସର୍ବଶ୍ରେଷ୍ଠ ତଥା ..."
397,10,33,अक्षराणामकारोऽस्मि द्वन्द्वः सामासिकस्य च।\n\n...,"ସଂସ୍କୃତ ଭାଷାରେ, ଯେ କୌଣସି ଅର୍ଦ୍ଧ ଅକ୍ଷର ସହିତ ‘ଅ’..."
610,17,24,तस्मादोमित्युदाहृत्य यज्ञदानतपःक्रियाः।प्रवर्त...,ଓଁ ଶବ୍ଦାଂଶଟି ନିରାକାର ବ୍ରହ୍ମଙ୍କର ପ୍ରତୀକାତ୍ମକ ପ୍...
29,1,34,आचार्याः पितरः पुत्रास्तथैव च पितामहाः।\n\nमात...,ଦ୍ରୋଣାଚାର୍ଯ୍ୟ ଏବଂ କୃପାଚାର୍ଯ୍ୟ ଅର୍ଜୁନଙ୍କ ଗୁରୁ ଥ...
30,1,35,एतान्न हन्तुमिच्छामि घ्नतोऽपि मधुसूदन।\n\nअपि ...,ଦ୍ରୋଣାଚାର୍ଯ୍ୟ ଏବଂ କୃପାଚାର୍ଯ୍ୟ ଅର୍ଜୁନଙ୍କ ଗୁରୁ ଥ...
412,11,6,पश्यादित्यान्वसून्रुद्रानश्िवनौ मरुतस्तथा।\n\n...,ଭଗବାନଙ୍କର ବିଶ୍ୱରୂପରେ ପୃଥିବୀ ପୃଷ୍ଠର ସମସ୍ତ ଐଶ୍ଚର...


# Clustering

## Max Pooling

In [35]:
kmeans_max = KMeans(n_clusters=60, random_state=0).fit(verse_embeddings_max)

In [36]:
def kmeans_max_cluster(query_embedding_list):
    label=kmeans_max.predict(query_embedding_list)[0]
    verse_index=[i for i in range(len(data)) if kmeans_max.labels_[i]==label]
    return verse_index

In [37]:
verse_index=kmeans_max_cluster(query_embedding_list)
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
24,1,29,सीदन्ति मम गात्राणि मुखं च परिशुष्यति।\n\nवेपथ...,ଯୁଦ୍ଧର ପରିଣତି ବିଷୟରେ ଚିନ୍ତା କରି ଅର୍ଜୁନ ବ୍ୟସ୍ତ ...
25,1,30,गाण्डीवं स्रंसते हस्तात्त्वक्चैव परिदह्यते।\n\...,ଯୁଦ୍ଧର ପରିଣତି ବିଷୟରେ ଚିନ୍ତା କରି ଅର୍ଜୁନ ବ୍ୟସ୍ତ ...
26,1,31,निमित्तानि च पश्यामि विपरीतानि केशव।\n\nन च श्...,ଯୁଦ୍ଧର ପରିଣତି ବିଷୟରେ ଚିନ୍ତା କରି ଅର୍ଜୁନ ବ୍ୟସ୍ତ ...
69,2,30,देही नित्यमवध्योऽयं देहे सर्वस्य भारत।\n\nतस्म...,ଅନେକ ସମୟରେ ଶିକ୍ଷା ଦାନ କରିବାକୁ ଯାଇ ଶ୍ରୀକୃଷ୍ଣ କୌ...
216,5,20,न प्रहृष्येत्प्रियं प्राप्य नोद्विजेत्प्राप्य ...,ଶ୍ଲୋକର ଏହି ଅଂଶ - ସୁଖରେ ଉତ୍‌ଫୁଲ୍ଲିତ ନ ହେବା ଓ ଦୁ...
245,6,20,यत्रोपरमते चित्तं निरुद्धं योगसेवया।\n\nयत्र च...,ଧ୍ୟାନର ବିଧି ଏବଂ ତା’ର ଚରମ ସ୍ଥିତି ସମ୍ପର୍କରେ ବର୍ଣ...
254,6,29,सर्वभूतस्थमात्मानं सर्वभूतानि चात्मनि।\n\nईक्ष...,"ଭାରତରେ ଦୀପାବଳି ପର୍ବରେ, ଦୋକାନମାନଙ୍କରେ ବିଭିନ୍ନ ଆ..."
269,6,44,पूर्वाभ्यासेन तेनैव ह्रियते ह्यवशोऽपि सः।\n\nज...,"ଥରେ ଆଧ୍ୟାତ୍ମିକ ଚେତନା ଅଙ୍କୁରିତ ହେଲେ, ତାହାର ମୂଳୋ..."
321,8,19,भूतग्रामः स एवायं भूत्वा भूत्वा प्रलीयते।\n\nर...,ବେଦରେ ଚାରିପ୍ରକାରର ପ୍ରଳୟ ସମ୍ବନ୍ଧରେ ଉଲ୍ଲେଖ କରାଯା...
341,9,11,अवजानन्ति मां मूढा मानुषीं तनुमाश्रितम्।\n\nपर...,ଉତ୍ତମ ଶିକ୍ଷକମାନେ ବେଳେବେଳେ ତାଙ୍କ ଛାତ୍ରମାନଙ୍କ ପ୍...


## Mean Pooling

In [38]:
kmeans_mean = KMeans(n_clusters=60, random_state=0).fit(verse_embeddings_mean)

In [39]:
def kmeans_mean_cluster(query_embedding_list):
    label=kmeans_mean.predict(query_embedding_list)[0]
    verse_index=[i for i in range(len(data)) if kmeans_mean.labels_[i]==label]
    return verse_index

In [40]:
verse_index=kmeans_mean_cluster(query_embedding_list)
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
60,2,21,वेदाविनाशिनं नित्यं य एनमजमव्ययम्।\n\nकथं स पु...,"ଆଧ୍ୟାତ୍ମିକତାରେ ଯେଉଁମାନେ ଉଚ୍ଚ କକ୍ଷରେ ଅବସ୍ଥାପିତ,..."
90,2,51,कर्मजं बुद्धियुक्ता हि फलं त्यक्त्वा मनीषिणः।\...,ଶ୍ରୀକୃଷ୍ଣ ଆସକ୍ତିରହିତ କର୍ମ ବିଷୟରେ ଆଲୋଚନା କ୍ରମରେ...
133,3,22,न मे पार्थास्ति कर्तव्यं त्रिषु लोकेषु किञ्चन।...,"ଆମ୍ଭେମାନେ କାହିଁକି କର୍ମ କରୁ, ତା’ର ପୃଷ୍ଠଦେଶରେ ଉଦ..."
140,3,29,प्रकृतेर्गुणसम्मूढाः सज्जन्ते गुणकर्मसु।\n\nता...,ଏଠାରେ ପ୍ରଶ୍ନ ଉଠିପାରେ ଯେ.ଯଦି ଆତ୍ମା ଗୁଣ ଏବଂ ସେ...
198,5,2,श्री भगवानुवाच\n\nसंन्यासः कर्मयोगश्च निःश्रेय...,ଏହି ଶ୍ଲୋକରେ ଶ୍ରୀକୃଷ୍ଣ କର୍ମ ସନ୍ୟାସ ଏବଂ କର୍ମଯୋଗ ...
200,5,4,सांख्ययोगौ पृथग्बालाः प्रवदन्ति न पण्डिताः।\n\...,ଶ୍ରୀକୃଷ୍ଣ ଏଠାରେ ସାଂଖ୍ୟ ଶବ୍ଦକୁ କର୍ମ ସନ୍ୟାସ ବା ଜ...
201,5,5,यत्सांख्यैः प्राप्यते स्थानं तद्योगैरपि गम्यते...,"ଅଧ୍ୟାତ୍ମିକ ସାଧନାରେ, ବାହ୍ୟ କ୍ରିୟା ଅପେକ୍ଷା, ମନୋଭ..."
210,5,14,न कर्तृत्वं न कर्माणि लोकस्य सृजति प्रभुः।\n\n...,"ଏହି ଶ୍ଲୋକରେ, ପ୍ରଭୁ ଶବ୍ଦଟି ଭଗବାନଙ୍କ ଉଦ୍ଦେଶ୍ୟରେ ..."
211,5,15,नादत्ते कस्यचित्पापं न चैव सुकृतं विभुः।\n\nअज...,ଭଗବାନ କୌଣସି ବ୍ୟକ୍ତିର ପୁଣ୍ୟ କର୍ମ ବା ପାପ କାର୍ଯ୍ୟ...
222,5,26,कामक्रोधवियुक्तानां यतीनां यतचेतसाम्।\n\nअभितो...,ଶ୍ଲୋକ ୫.୨ରେ ବର୍ଣ୍ଣନା କରାଯାଇଛି ଯେ କର୍ମଯୋଗ ଅଧିକା...


## Whole

In [41]:
kmeans_whole = KMeans(n_clusters=60, random_state=0).fit(verse_embeddings_whole)

In [42]:
def kmeans_whole_cluster(query_embedding_list):
    label=kmeans_whole.predict(query_embedding_list)[0]
    verse_index=[i for i in range(len(data)) if kmeans_whole.labels_[i]==label]
    return verse_index

In [43]:
verse_index=kmeans_whole_cluster(query_embedding_list)
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
12,1,15,पाञ्चजन्यं हृषीकेशो देवदत्तं धनंजयः।\n\nपौण्ड्...,“ହୃଷିକେଶ” ଶବ୍ଦଟି ଶ୍ରୀକୃଷ୍ଣଙ୍କ ଉଦ୍ଦେଶ୍ୟରେ ବ୍ୟବହ...
200,5,4,सांख्ययोगौ पृथग्बालाः प्रवदन्ति न पण्डिताः।\n\...,ଶ୍ରୀକୃଷ୍ଣ ଏଠାରେ ସାଂଖ୍ୟ ଶବ୍ଦକୁ କର୍ମ ସନ୍ୟାସ ବା ଜ...
215,5,19,इहैव तैर्जितः सर्गो येषां साम्ये स्थितं मनः।\n...,"ଶ୍ରୀକୃଷ୍ଣ ଏଠାରେ ସାମ୍ୟେ ଶବ୍ଦକୁ, ପୂର୍ବ ଶ୍ଲୋକରେ ବ..."
221,5,25,लभन्ते ब्रह्मनिर्वाणमृषयः क्षीणकल्मषाः।\n\nछिन...,"ପୂର୍ବ ଶ୍ଲୋକରେ, ଶ୍ରୀକୃଷ୍ଣ ସେହି ମହାପୁରୁଷମାନଙ୍କ ବ..."
243,6,18,यदा विनियतं चित्तमात्मन्येवावतिष्ठते।\n\nनिःस्...,ସାଧକର ଯୋଗାଭ୍ୟାସ କେତେବେଳେ ସମ୍ପୂର୍ଣ୍ଣ ହୁଏ? ଯେତେବ...
262,6,37,अर्जुन उवाच\n\nअयतिः श्रद्धयोपेतो योगाच्चलितमा...,ଭଗବତ୍ ପ୍ରାପ୍ତିର ଯାତ୍ରା ଶ୍ରଦ୍ଧାଠାରୁ ଆରମ୍ଭ ହୁଏ ।...
269,6,44,पूर्वाभ्यासेन तेनैव ह्रियते ह्यवशोऽपि सः।\n\nज...,"ଥରେ ଆଧ୍ୟାତ୍ମିକ ଚେତନା ଅଙ୍କୁରିତ ହେଲେ, ତାହାର ମୂଳୋ..."
287,7,15,न मां दुष्कृतिनो मूढाः प्रपद्यन्ते नराधमाः।\n\...,ଭଗବାନଙ୍କର ଶରଣାଗତ ହେଉ ନ ଥିବା ଚାରି ପ୍ରକାର ଲୋକଙ୍କ...
289,7,17,तेषां ज्ञानी नित्ययुक्त एकभक्ितर्विशिष्यते।\n\...,"ଯେଉଁମାନେ ଦୁଃଖରେ ସଂତପ୍ତ ହୋଇ, ସାଂସାରିକ ବସ୍ତୁ ପାଇ..."
302,7,30,साधिभूताधिदैवं मां साधियज्ञं च ये विदुः।\n\nप्...,"ପରବର୍ତ୍ତୀ ଅଧ୍ୟାୟରେ, ଶ୍ରୀକୃଷ୍ଣ କହିବେ ଯେ, ଯେଉଁ ମ..."
